https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization

In [2]:
import functools
import os
CWD = os.getcwd()
from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import shutil
import tensorflow as tf
import keras
import tensorflow_hub as hub

print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))

2024-01-05 20:19:51.149929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TF Version:  2.12.0
TF Hub version:  0.15.0
Eager mode enabled:  True
GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-01-05 20:19:52.592302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-05 20:19:52.617033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-05 20:19:52.617101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# Load TF Hub module.

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

2024-01-05 20:19:54.680046: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-05 20:19:54.680126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-05 20:19:54.680155: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-05 20:19:55.022733: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-05 20:19:55.022789: I tensorflow/compile

In [4]:
def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(PATH, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = tf.io.decode_image(
      tf.io.read_file(PATH),
      channels=3, dtype=tf.float32)[tf.newaxis, ...]
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()

In [14]:
def stylize(content_image, style_image):
    outputs = hub_module(content_image, style_image)
    stylized_image = outputs[0]
    # Stylize content image with given style image.
    # This is pretty fast within a few milliseconds on a GPU.
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    # print(outputs)
    stylized_image = outputs[0]
    # Visualize input images and the generated stylized image.
    # show_n([content_image, style_image, stylized_image], titles=['Original content image', 'Style image', 'Stylized image'])
    stylized_image = tf.squeeze(stylized_image)
    return stylized_image

If I run the code below several times, the random randint function should ensure that I get a whole bunch of weird lookin images with different skin tones

In [16]:
import random
for content_img in os.listdir(f'{CWD}/content_images'):
    i = random.randint(0,43)
    style_img = os.listdir(f'{CWD}/style_images')[i]
    style_image = load_image(f"{CWD}/style_images/{style_img}", (224,224))
    content_image = load_image(f'{CWD}/content_images/{content_img}', (256, 256))
    style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
    style_img = style_img[0:4]
    content_img = content_img[0:12]
    keras.utils.save_img(f'{CWD}/stylized/{content_img}_stylized_with_{style_img}.jpg', stylize(content_image, style_image))
    # show_n([content_image, style_image], ['Content image', 'Style image'])